In [1]:
from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential, load_model
from keras.layers import Dropout, Flatten, Dense, Conv2D
from keras.layers.core import Flatten, Dense, Dropout
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras.utils import multi_gpu_model
from sklearn.utils import class_weight
# from tensorflow.keras.layers import Dense, Flatten, Dropout, MaxPooling2D
# from keras.optimizers import SGD
import cv2, numpy as np
import tensorflow as tf

In [6]:
img_width, img_height = 224, 224

top_model_weights_path = '128_128_2_2_4classes.h5'#bottleneck_fc_model_multi_40epoch_6class_2lyr.h5'
save_weights = 'fin_ep30_bt64_bed_book_ch_desk_sof_tb_4class.h5'
train_data_dir = 'pix3d-ML/train'
validation_data_dir = 'pix3d-ML/validation'
test_data_dir = 'pix3d-ML/test'
botfet_train = 'bottleneck_features_train.npy'
botfet_validation = 'bottleneck_features_validation.npy'
num_classes = 4
# t = [1049,914,1097,1245,1056,1743]
# z = [249,191,254,244,244,354] #sum must be div by 16
t = [920,1097,1056,1743]#1049,1245,
z = [192,254,244,350] #sum must be div by 16 #249,244,
nb_train_samples = sum(t)
nb_validation_samples = sum(z)
epochs = 30
batch_size = 64

In [3]:
# compile the model with a SGD/momentum optimizer
# and a very slow learning rate.
# prepare data augmentation configuration
train_datagen = ImageDataGenerator(
    rescale=1. / 255)
#     shear_range=0.3,
#     zoom_range=0.2,
#     horizontal_flip=True,
#     rotation_range=40,
#     width_shift_range=0.2,
#     height_shift_range=0.2,
#     fill_mode='nearest')

test_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical')

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical')

Found 4816 images belonging to 4 classes.
Found 1040 images belonging to 4 classes.


In [15]:
#Take care on unbalanced class
# y_train = np.load(open('label_train.npy','rb'))
y_train = np.array([0]*z[0] + [1]*z[1] + [2]*z[2] + [3]*z[3])
# print(y_train)
class_weights = class_weight.compute_class_weight('balanced',
                                                 np.unique(y_train),
                                                 y_train)

cl = {}
for i in range(len(class_weights)):
    cl[i] = class_weights[i]
print(cl)
# cl={0:1.12869399,1:1.29540481,2:1.0793072,3:0.95100402,4:1.12121212,5:0.67928858}

{0: 1.3541666666666667, 1: 1.0236220472440944, 2: 1.0655737704918034, 3: 0.7428571428571429}


In [7]:
# import os
# os.environ["CUDA_VISIBLE_DEVICES"]="0,1,2,3"
# server = tf.train.Server.create_local_server()
# sess = tf.Session(server.target)

# from keras import backend as K
# K.set_session(sess)
strategy = tf.distribute.MirroredStrategy()
print('Number of devices: {}'.format(strategy.num_replicas_in_sync))
with strategy.scope():
    # build the VGG16 network
    model = applications.VGG16(weights='imagenet', include_top=False,input_shape = (224,224,3))
    print('Model saved.')
    # build a classifier model to put on top of the convolutional model
    top_model = Sequential()
    print(model.output_shape[1:],model.output_shape)
#     top_model.add(Flatten(input_shape=model.output_shape[1:]))
#     top_model.add(Dense(256, activation='relu'))
#     top_model.add(Dropout(0.5))
#     top_model.add(Dense(num_classes, activation='softmax'))
    top_model.add(Flatten(input_shape=model.output_shape[1:]))
    top_model.add(Dense(128, activation='relu'))
    top_model.add(Dropout(0.2))
    top_model.add(Dense(128, activation='relu'))
    top_model.add(Dropout(0.3))
    top_model.add(Dense(num_classes, activation='softmax'))

# note that it is necessary to start with a fully-trained
# classifier, including the top classifier,
# in order to successfully do fine-tuning
    top_model.load_weights(top_model_weights_path)

# add the model on top of the convolutional base
    mb = Sequential()
    mb.add(model)
    mb.add(top_model)
    # model.add(top_model)
    model = mb
#     for layer in model.layers[:25]:
#         layer.trainable = False
    
    model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.SGD(lr=1e-5, momentum=0.9),
              metrics=['accuracy'])


model.fit(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=nb_validation_samples // batch_size,
    class_weight=cl)

model.save_weights(save_weights)

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1')
Number of devices: 2
Model saved.
(7, 7, 512) (None, 7, 7, 512)
Epoch 1/30
INFO:tensorflow:batch_all_reduce: 32 all-reduces with algorithm = nccl, num_packs = 1
INFO:tensorflow:batch_all_reduce: 32 all-reduces with algorithm = nccl, num_packs = 1
75/75 [==============================] - 15s 203ms/step - loss: 0.2122 - accuracy: 0.9329 - val_loss: 0.1415 - val_accuracy: 0.9518
Epoch 2/30
75/75 [==============================] - 14s 181ms/step - loss: 0.1581 - accuracy: 0.9455 - val_loss: 0.1090 - val_accuracy: 0.9652
Epoch 3/30
75/75 [==============================] - 14s 183ms/step - loss: 0.1574 - accuracy: 0.9478 - val_loss: 0.0983 - val_accuracy: 0.9703
Epoch 4/30
75/75 [==============================] - 13s 179ms/step - loss: 0.1516 - accuracy: 0.9499 - val_loss: 0.0927 - val_accuracy: 0.9693
Epoch 5/30
75/75 [=========================

In [8]:
def only_load():
    model = applications.VGG16(weights='imagenet', include_top=False,input_shape = (224,224,3))
    print('Model saved.')
    # build a classifier model to put on top of the convolutional model
    top_model = Sequential()
    print(model.output_shape[1:],model.output_shape)
    top_model.add(Flatten(input_shape=model.output_shape[1:]))
    top_model.add(Dense(256, activation='relu'))
    top_model.add(Dropout(0.5))
    top_model.add(Dense(num_classes, activation='softmax'))

    # note that it is necessary to start with a fully-trained
    # classifier, including the top classifier,
    # in order to successfully do fine-tuning
    top_model.load_weights(top_model_weights_path)

    # add the model on top of the convolutional base
    mb = Sequential()
    mb.add(model)
    mb.add(top_model)
    # model.add(top_model)
#     model = mb
    mb.load_weights('fin_ep50_bt64_bed_book_ch_desk_sof_tb.h5')
    return mb

In [8]:
labels = (train_generator.class_indices)
print(labels)

{'bookcase': 0, 'chair': 1, 'sofa': 2, 'table': 3}


In [9]:
import os
from PIL import Image,ImageFilter

# model = only_load()

pos = [0]*num_classes #pos,neg: match and mismatch for each class
neg = [0]*num_classes
comp = [pos*num_classes]
#what is the highest wrong class for each class
mapp = {'bookcase': 0, 'chair': 1, 'sofa': 2, 'table': 3}
for r,d,f in os.walk(test_data_dir):
    for files in f:
        rr = r.split('/')
        idr = mapp[rr[2]]
        
        image2 = np.asarray(Image.open(r+'/'+files))
        image2 = image2[:,:,0:3]
#             print(image2.shape)
        image2 = np.reshape(image2,[1,224,224,3])
        classes = model.predict(image2)[0]
#             classes = classes[0]
#             print(classes)
        if sorted(classes)[-2]!=0:
            sr = [sorted(classes)[-2],sorted(classes)[-1]]
        else:
            sr = [sorted(classes)[-1]]
#         print('cls:',classes,sr,idr,rr)
        
        if classes[idr] in sr:
            pos[idr]+=1
            if classes[idr]!=sr[-1]:
#                 print(classes,np.where(classes==sr[-1])[0][0])
                comp[idr][np.where(classes==sr[-1])[0][0]]+=1
        else:
            neg[idr]+=1
                
        
print('done')

done


In [12]:
print(pos,neg,comp)

[107, 82, 312, 92, 0, 0] [31, 22, 81, 20, 0, 0] [[0, 3, 0, 1, 0, 0], [0, 0, 3, 5, 0, 0], [0, 32, 0, 9, 0, 0], [0, 8, 2, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0]]


In [14]:
for en,item in enumerate(pos):
    print(item/(item+neg[en]))
    #128,128,2,2 all layer unfrozen

0.34146341463414637
0.6231884057971014
0.6923076923076923
0.22580645161290322
0.72264631043257
0.8839285714285714


In [11]:
for en,item in enumerate(pos):
    print(item/(item+neg[en]))
    

0.7753623188405797
0.7884615384615384
0.7938931297709924
0.8214285714285714


ZeroDivisionError: division by zero

In [21]:
for en,item in enumerate(pos):
    print(item/(item+neg[en]))
    # 128 128 2 2 layer frozen

0.4146341463414634
0.5434782608695652
0.46153846153846156
0.3118279569892473
0.8193384223918575
0.8660714285714286


In [50]:
for en,item in enumerate(pos):
    print(item/(item+neg[en]))

0.6676829268292683
0.5
0.46153846153846156
0.17204301075268819
0.5928753180661578
0.625


In [8]:
for en,item in enumerate(pos):
    print(item/(item+neg[en]))
    #128,128,2,2 all layer unfreeze e-5,40epoch

0.3170731707317073
0.6086956521739131
0.7019230769230769
0.22580645161290322
0.6666666666666666
0.8928571428571429
